In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('../input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')
validation_data = pd.read_csv('../input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')

In [3]:
print(train_data.info)
print(validation_data.info)

<bound method DataFrame.info of                                               id  \
0       0001d1afc246a7964130f43ae940af6bc6c57f01   
1       0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2       00027e965c8264c35cc1bc55556db388da82b07f   
3       0002c17436637c4fe1837c935c04de47adb18e9a   
4       0003ad6ef0c37534f80b55b4235108024b407f0b   
...                                          ...   
287108  fffdfb56fdf1a12d364562cc2b9b1d4de7481dee   
287109  fffeecb8690b85de8c3faed80adbc7a978f9ae2a   
287110  ffff5231e4c71544bc6c97015cdb16c60e42b3f4   
287111  ffff924b14a8d82058b6c1c5368ff1113c1632af   
287112  ffffd563a96104f5cf4493cfa701a65f31b06abf   

                                                  article  \
0       By . Associated Press . PUBLISHED: . 14:11 EST...   
1       (CNN) -- Ralph Mata was an internal affairs li...   
2       A drunk driver who killed a young woman in a h...   
3       (CNN) -- With a breezy sweep of his pen Presid...   
4       Fleetwood are the only team st

No preprocessing was done to the article and highlights itself as news can lose some of its original message when all the letter is in lowercase

take only 20.000 rows from training dataset and 900 rows from validation and testing dataset due to computation limitation,

In [4]:
train_data = train_data.sample(n=20000, random_state=123)
train_data = train_data.reset_index()
validation_data = validation_data.sample(n=900, random_state=312)
validation_data = validation_data.reset_index()

In [5]:
pd.set_option('display.max_colwidth',None)
train_data.head(1)

,index,id,article,highlights
0,252682,d303b222282ba8870fe997c030f8582ef8809ab8,"A group of Brazilian convicts who tunnelled out of jail to freedom filmed the whole escape on a mobile phone ... before posting the videos online to taunt their hoodwinked guards. In a daring breakout that echoed 1994 Hollywood blockbuster The Shawshank Redemption, 13 prisoners dug a small hole under the wall between their cell and the prison yard in Rio Verde, Goias State, before darting to freedom under cover of darkness. But unlike the famous movie, starring Tim Robbins and Morgan Freeman, which is set in the 1940s, their escape had very modern twist. Rather than focus purely on getting out of jail without detection, they felt compelled to record the whole experience on a mobile phone and post it on the internet for all to see. Scroll down for video . Brazen: The thirteen prisoners filmed their escape on mobile phones, laughing as they did so, before later posting the video online . Like a movie: In a daring breakout that echoed 1994 Hollywood blockbuster The Shawshank Redemption, the prisoners dug a small hole under the wall that separated their cell and the prison yard in Rio Verde, Goias State, before darting to freedom under cover of darkness . Police deputy Jacqueline Camargo Machado admitted that the prisoners' cells had not been inspected for two months, blaming staff shortages. She said: 'The guard that was there on the day was looking after other things most of the time. 'We now know that there had not been an inspection of the cell for almost 60 days. They were able to dig a tunnel and it was easy to escape after that.' Ezequias Gonçalves Miranda and William Pereira Da Mata were two of the crooks that tunnelled out of jail to freedom. It is unclear if they appeared in the video . More than Freeman: It was a daring breakout that echoed 1994 Hollywood blockbuster The Shawshank Redemption, starring Morgan Freeman (left) and Tim Robbins (right) - but with a very modern twist . The prison guard on duty has been suspended for not managing to foil the secret plot. A manhunt has been launched resulting in the capture of two men. The other 11 remain at large. It is not the first escape from Brazilian jail. A year ago 54 prisoners staged a mass escape after a riot at the Fundação Casa youth detentionfacility in Sao Paulo, which houses more than 100 detainees. Dozens of the under-age offenders set fire to their mattresses, sparking a blaze in the prison's central yard. Helicopter images showed plumes of smoke rising from the detention centre. Video footage also showed prisoners climbing over the prison walls and escaping down trees. Nearly 30 staff members were held hostage during the riot and the facility's director was admitted to hospital with cuts to his head and bruised ribs.","13 prisoners escaped from Rio Grande prison after digging hole under wall .\nThe escape had shades of 1994 blockbuster The Shawshank Redemption .\nBut in a modern twist, they filmed escape on smuggled mobile phones .\nThey then posted the video online which showed them goading guards ."


In [6]:
!pip install datasets transformers rouge-score nltk -q
!pip install spacy-transformers
!pip install spacy

     |████████████████████████████████| 51 kB 88 kB/s             
     |████████████████████████████████| 1.1 MB 1.1 MB/s            


In [7]:
!pip install torch

In [8]:
import torch
import datasets
from datasets import Dataset
from datasets import load_metric
import transformers 
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer

In [9]:
from transformers import AutoTokenizer

model_chkpt_selected ='sshleifer/distilbart-xsum-9-6'
    
tokenizer = AutoTokenizer.from_pretrained(model_chkpt_selected)

pad_on_right = tokenizer.padding_side == "right"

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Tokenizer example 

In [10]:
tokenizer("Hello, this one sentence!")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
max_input_length = 800 #max article words to be summarized
max_target_length = 64 #max highlights words in the summary

In [12]:
#tokenize the dataset for Transformers model
def preprocess_function(samples):
    inputs = ['summarize:' + doc for doc in samples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(samples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
train_ds = Dataset.from_pandas(train_data)
valid_ds = Dataset.from_pandas(validation_data)

In [14]:
tokenized_train = train_ds.map(preprocess_function, batched = True)
tokenized_validation = valid_ds.map(preprocess_function, batched = True)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

## Fine tuning 

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_chkpt_selected)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

In [17]:
batch_size = 6 

collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [19]:
import gc
gc.collect()

717

In [20]:
# determine the device we will be using for training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))

[INFO] training using Tesla P100-PCIE-16GB


In [21]:
torch.cuda.empty_cache()

In [22]:
torch.cuda.set_per_process_memory_fraction(1.0)

In [23]:
torch.cuda.memory_reserved()

0

In [24]:
model_name = model_chkpt_selected.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-CNN_DailyMail",
    evaluation_strategy = "epoch",
    learning_rate=5e-7, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01, 
    save_total_limit=3,
    num_train_epochs=1, 
    predict_with_generate=True,
    fp16=True,
    save_strategy = "no"
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    data_collator=collator,
    tokenizer=tokenizer,
)

Using amp fp16 backend


In [26]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: index, id, highlights, article.
***** Running training *****
  Num examples = 20000
  Num Epochs = 1
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 3334
/opt/conda/lib/python3.7/site-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss
1,2.541400,2.217775


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: index, id, highlights, article.
***** Running Evaluation *****
  Num examples = 900
  Batch size = 6


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3334, training_loss=2.8744458505759787, metrics={'train_runtime': 3036.5836, 'train_samples_per_second': 6.586, 'train_steps_per_second': 1.098, 'total_flos': 2.220149108304691e+16, 'train_loss': 2.8744458505759787, 'epoch': 1.0})

In [27]:
trainer.save_model("DistilBART-Large_XSum_CNN_20k")

Saving model checkpoint to DistilBART-Large_XSum_CNN_20k
Configuration saved in DistilBART-Large_XSum_CNN_20k/config.json
Model weights saved in DistilBART-Large_XSum_CNN_20k/pytorch_model.bin
tokenizer config file saved in DistilBART-Large_XSum_CNN_20k/tokenizer_config.json
Special tokens file saved in DistilBART-Large_XSum_CNN_20k/special_tokens_map.json
